This code is used to generate synthetic spatial transcriptomics data.

In [1]:
import bisect
import random
import itertools
from math import exp

import pandas as pd
import numpy as np
from numpy.random import default_rng
from scipy.spatial import Voronoi, voronoi_plot_2d, cKDTree

In [2]:
import pandas as pd 
for i in range(100):
    
    df = pd.read_csv(f'/Users/kate/Research/topact/output/synthetic/synthetic{i}/reads.csv')
    
    df.to_csv(f'../syntheticdata/expression/reads_{i}.csv')

In [2]:
def readfile(filename):
    with open(filename, 'r') as f:
        return [line.rstrip() for line in f.readlines()]

In [3]:
# Cell type counts in snRNA-seq reference

counts={'CCD': 3086,
        'DCT': 6490,
        'ENDO': 3850,
        'IMMUNE': 1013,
        'PODS':  215,
        'PT': 18109,
        'TAL': 8081,
        'UROTHELIUM': 161,
        'VSM': 3127
        }

# Total number of cells in snRNA-seq reference
total = sum(counts.values())

# List of cell types sorted alphabetically
CELLTYPES = sorted(list(counts.keys()))

# List of cell type proportions
PROPORTIONS = [counts[x]/total for x in CELLTYPES]

# Cell types sorted by proportion
CELLTYPES = [x for _, x in sorted(zip(PROPORTIONS, CELLTYPES), reverse=True)]
PROPORTIONS = sorted(PROPORTIONS, reverse=True)

# Thresholds for when random number in [0,1] is assigned a given cell type
THRESHOLDS = PROPORTIONS.copy()
for i in range(1, len(THRESHOLDS)):
    THRESHOLDS[i] += THRESHOLDS[i-1]
    
def get_classification(value):
    return CELLTYPES[bisect.bisect_right(THRESHOLDS, value)]
    
# Average expressions of each cell type in snRNA-seq reference
AVERAGE_EXPRESSIONS = {ct: np.loadtxt(f'./averages/{ct}-average.txt') for ct in CELLTYPES}

# List of genes
GENES = readfile('./genes.txt')

NUM_GENES = len(GENES)
NUM_CELLTYPES = len(CELLTYPES)

In [4]:
# This ordering is used later for plotting

print(CELLTYPES)

['PT', 'TAL', 'DCT', 'ENDO', 'VSM', 'CCD', 'IMMUNE', 'PODS', 'UROTHELIUM']


In [7]:
# Generate random point cloud
def get_points(num_points=2500, dimensions=[1000,1000]):
    x = np.random.uniform(low=0, high=dimensions[1], size=num_points)
    y = np.random.uniform(low=0, high=dimensions[0], size=num_points)
    
    return list(zip(x, y))

# Generate random cell type classifications
def get_classifications(num_points):
    values = list(np.random.uniform(size=num_points))
    return list(map(get_classification, values))

# Generate gene expression
def generate_readings(spots, alpha, zeros=0):
    ea = exp(-alpha)
    rng = default_rng()
    lams = np.vstack([AVERAGE_EXPRESSIONS[ct] for ct in spots]) * ea
    vals = rng.poisson(lams, (len(spots), NUM_GENES))
    expr = np.vstack([vals[i] for i, _ in enumerate(spots)])

    # Model zero inflation
    if zeros > 0:
        indices = rng.choice(np.arange(len(spots)), size=int(len(spots) * zeros), replace=False)
        expr[indices] = 0

    return expr
            
# Convert expression to dataframe rows
def convert_to_rows(coords, expression_matrix):
    
    reads = zip(*np.where(expression_matrix > 0))
    
    for i, j in reads:
        x, y = coords[i]
        gene = GENES[j]
        
        yield {'geneID': gene, 'x': x, 'y': y, 'MIDCounts': expression_matrix[i,j]}
        
# Create dataframe of gene expression
def create_expression_df(spots, spot_types, alpha, zeros=0):
    return pd.DataFrame(convert_to_rows(spots, generate_readings(spot_types, alpha, zeros)))

    
# Run entire process of generating synthetic data
def get_everything(num_cells = 2500, dimensions = (1000,1000), alpha=7, zeros=0):
    cells = get_points(num_cells, dimensions)
    celltypes = get_classifications(num_cells)

    spots = np.array(list(itertools.product(range(dimensions[0]), range(dimensions[1]))))

    voronoi_kdtree = cKDTree(cells)

    spot_dist, spot_regions = voronoi_kdtree.query(spots, k=1)
    spot_types = list(map(lambda x: celltypes[x], spot_regions))

    df = create_expression_df(spots, spot_types, alpha, zeros)
    
    spot_cells = np.empty(dimensions)
    for ((x,y), cell) in zip(spots, spot_regions):
        spot_cells[x,y] = cell
        
    ct_grid = np.empty(dimensions)
    for ((x,y), ct) in zip(spots, spot_types):
        ct_grid[x,y] = CELLTYPES.index(ct)
        
    return cells, celltypes, spot_cells, ct_grid, df

Here is a small example of how synthetic data are generated. Be warned that generating large datasets can use a lot of memory.

In [8]:
d = 50 # For actual data, d=500
num_cells = 6 # For actual data, num_cells=625

alpha=7.3 # This is in fact -log(alpha) for the alpha defined in thepa per

total_reads = 0

# Example generating a set of synthetic data. Data used in analysis can be found in a subdirectory.

cells, celltypes, spot_cells, ct_grid, df = get_everything(num_cells, (d,d), alpha=alpha, zeros=0.2)

In [10]:
# List of cell coordinates
print(cells)

[(49.91038297521845, 26.296220871763847), (2.1895783438592273, 32.432760530575806), (13.228270082728322, 9.71806710212893), (11.130368172892851, 0.5839765818611897), (14.51342159038832, 8.936062602554696), (15.033114558422294, 4.545950433175233)]


In [11]:
# List of cell types for each cell
print(celltypes)

['PT', 'CCD', 'PT', 'ENDO', 'PT', 'VSM']


In [12]:
# Array showing cell assigned to each spot
print(spot_cells)

[[3. 3. 3. ... 1. 1. 1.]
 [3. 3. 3. ... 1. 1. 1.]
 [3. 3. 3. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
# Array showing cell type assigned t each spot
print(ct_grid)

[[3. 3. 3. ... 5. 5. 5.]
 [3. 3. 3. ... 5. 5. 5.]
 [3. 3. 3. ... 5. 5. 5.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
# Data frame showing generated expression
# MIDCounts = # transcript reads
print(df)

             geneID   x   y  MIDCounts
0             Krit1   0   3          1
1     4932438A13Rik   0   7          1
2            Luc7l3   0   8          1
3            Eef1a1   0   8          1
4              Amy1   0   8          1
...             ...  ..  ..        ...
4175         Polr3b  49  47          1
4176       Epm2aip1  49  47          1
4177          Knop1  49  48          1
4178          Oscp1  49  48          1
4179            Nlk  49  49          1

[4180 rows x 4 columns]
